# Automatic Syllabus Classification

In [1]:
from os import environ
from google.generativeai import configure, GenerativeModel
from requests import get

from syllabus_classificaiton.agents import agents
from syllabus_classificaiton.prompt import prompt

/Users/michaelkohlegger/Documents/GitHub/syllabus-classification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
competences = get("https://raw.githubusercontent.com/mckoh/digicomp2-2/refs/heads/main/competences.json").content

In [3]:
lv_description = {
    "name": "MACHINE LEARNING & DEEP LEARNING",
    "objectives": "- Die Studierenden kennen Werkzeuge (z.B. Bibliotheken, Cloud Plattformen oder Softwarewerkzeuge), mit deren Hilfe Machine Learning unterstützt werden kann. - Die Studierenden können die erarbeiteten Werkzeuge hinsichtlich ihrer Eignung für konkrete Problemstellungen vergleichen. - Die Studierenden können Ende-zu-Ende Machine Learning Projekte konzipieren. - Die Studierenden können Ende-zu-Ende Machine Learning Projekte selbstständig durchführen."
}

In [4]:
prompt = prompt.replace("**name**", lv_description["name"])
prompt = prompt.replace("**objectives**", lv_description["objectives"])

## Model Setup

In [5]:
configure(api_key=environ["GEN_AI_KEY"])

model = GenerativeModel("models/gemini-1.5-flash")

## Collect Responses from Agents

In [7]:
responses = []

for agent in agents:
    chat = model.start_chat(history=[])

    _ = chat.send_message(f"Hier stelle ich dir eine formale Beschreibung des DigiComp 2.2 Kompetenzrasters zur Verfügung: {competences}")

    prompt = prompt.replace("**role**", agent["name"])
    prompt = prompt.replace("**description**", agent["description"])

    response = chat.send_message(content=prompt, stream=False)
    responses.append(response.text)

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
]

In [8]:
for i, response in enumerate(responses):
    with open(f"ergebnisse/agent_{i}.md", "w") as text_file:
        text_file.write(response)